In [ ]:
from typing import List
from random import choices
import numpy as np
import json
import pandas as pd

In [ ]:

def bootstrap(results: List[int], B: int = 10000, confidence_level: int = 0.95) -> int:

    """
    helper function for providing confidence intervals for sentiment tool
    """

    # compute lower and upper significance index
    critical_value = (1-confidence_level)/2
    lower_sig = 100*critical_value
    upper_sig = 100*(1-critical_value)
    data = []
    for p in results:
        data.append(p)

    sums = []
    # bootstrap resampling loop
    for b in range(B):
        choice = choices(data, k=len(data))
        choice = np.array(choice)
        inner_sum = np.sum(choice)/len(choice)
        sums.append(inner_sum)

    percentiles = np.percentile(sums, [lower_sig, 50, upper_sig])

    lower = percentiles[0]
    median = percentiles[1]
    upper = percentiles[2]

    e_bar = ((median- lower) + (upper - median))/2
    return e_bar, median, percentiles

In [ ]:
with open('robertas-log-calibragpt-full-mar-14.json', 'r') as f:
    data = json.load(f)

model_name = data['model_name']

model_name

In [ ]:
false_facts_itrs = []
false_facts = []
true_facts_itrs = []

ratios = {}
ratios['differences'] = []
ratios['facts'] = []

for itr, fact in enumerate(data["score_dict_full"][model_name.lower()]):
    for x, y in fact.items():
        if y['p_true > p_false'] != "True":
            false_facts_itrs.append(itr)
            false_facts.append([x, y])
            ratios['differences'].append(y['p_true - p_false'])
            ratios['facts'].append(x)
        elif y['p_true > p_false'] == "True":
            true_facts_itrs.append(itr)

In [ ]:
# make a results list compatible with the bootstrap:

results_false = [0] * len(false_facts_itrs)
results_true = [1] * len(true_facts_itrs)

In [ ]:
results = results_false + results_true

In [ ]:
len(results)  # should be 72370

In [ ]:
# create bootstrap estimates from logs

# calculate percentage with this to check
np.sum(results)/len(results)

In [ ]:
bootstrap(results)


In [ ]:
# order results by p_true - p_false

pd.DataFrame.from_dict(ratios).sort_values(by='differences').head(20)
#pd.DataFrame.from_dict(ratios).sort_values(by='differences').head(20).to_csv('test.csv')

In [ ]:
# @TimSchott - see second to last entry above